웹 서버 브라우저와의 안전한 통신을 위해 HTTPS를 사용하기 위해서는 SSL 인증서가 필요합니다. 

https://letsencrypt.org 을 이용하면, 개인용이나, 개발용으로 싱글 DNS에 대해서 SSL 인증서를 발급받는 것을 무료로 진행할 수 있습니다.

Let's encrypt에서 제공하는 퉁을 이용해 SSL 인증서를 발급 받아보고, pem 확장자의 인증 파일을 SSL 오프로드를 위해 사용한 azure application gateway에서 사용할 pfx 확장자 인증 파일로 변환해 봅니다.

<!-- TEASER_END -->

# letsencrypt tool install

```
> sudo apt update -y && sudo apt install letsencrypt -y
```

설치 후 실행 시 아래와 같은 에러가 발생하면,

```
Traceback (most recent call last):
...
(중략)
...
  File "/usr/share/python-wheels/urllib3-1.13.1-py2.py3-none-any.whl/urllib3/contrib/pyopenssl.py", line 54, in <module>
  File "/usr/lib/python2.7/dist-packages/OpenSSL/__init__.py", line 8, in <module>
    from OpenSSL import rand, crypto, SSL
  File "/usr/lib/python2.7/dist-packages/OpenSSL/SSL.py", line 118, in <module>
    SSL_ST_INIT = _lib.SSL_ST_INIT
AttributeError: 'module' object has no attribute 'SSL_ST_INIT'
```

아래 명령으로 Python SSL 모듈을 업그레이드 해줍니다.

```
> sudo python -m easy_install --upgrade pyOpenSSL
```

# create manual cert keys

80번 포트를 사용하고 있는 프로그램이 있다면, 먼저 중지해 줍니다.

letsencrypt 에서 80 포트를 이용해 자동 인증 후 key 파일을 생성 해줍니다.

```
> sudo letsencrypt certonly --standalone -d yourdomain.dns.com

  signer = key.signer(self.padding, self.hash)

IMPORTANT NOTES:
 - Congratulations! Your certificate and chain have been saved at
   /etc/letsencrypt/live/dev.stylegrab.style/fullchain.pem. Your cert
   will expire on 2018-10-18. To obtain a new version of the
   certificate in the future, simply run Let's Encrypt again.
 - If you like Let's Encrypt, please consider supporting our work by:

   Donating to ISRG / Let's Encrypt:   https://letsencrypt.org/donate
   Donating to EFF:                    https://eff.org/donate-le

```

생성된 키 파일들은 아래 디렉토리에 생성됩니다. 

```
> ls -al /etc/letsencrypt/archive/yourdomain.dns.com/

-rw-r--r-- 1 root root 2167  7월 21 04:23 cert1.pem
-rw-r--r-- 1 root root 1647  7월 21 04:23 chain1.pem
-rw-r--r-- 1 root root 3814  7월 21 04:23 fullchain1.pem
-rw-r--r-- 1 root root 1704  7월 21 04:23 privkey1.pem
```

# convert pem to pfx file

때에 따라서는 http 에 ssl적용을 위해서 pfx 파일이 필요할 때도 있습니다.

이럴때는 아래 명령으로 pfx file 을 생성해서 사용하시면 됩니다.

```
> openssl pkcs12 -export -out cert.pfx -inkey privkey1.pem -in cert1.pem -certfile chain1.pem
```